<a href="https://colab.research.google.com/github/leiayoung/personal_health_project/blob/main/personal_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A fun little side project to analyze data I've gathered on my personal health over the last couple of years. My first step was to create some visualizations in Tableau. This answered many questions but posed more. I'm now coming over to Python to manipulate my data more directly (dealing with missing values, offsetting dates, ect.) then ulitimately I want to develop some models and make future predictions.

In [16]:
# libraries for this project
import numpy as np 
import pandas as pd  
from datetime import datetime, timedelta
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [11]:
# Connect data from Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id_n = '1-10-Ll-Njrnj3xyI9x_V_H72R0hx3zDl' # connecting to my nutrition data
file_id_w = '1SsirP4TgYJE3hraih0wUpWYcgr3Hj1Hc' # connecting my weight data
download_n = drive.CreateFile({'id': file_id_n})
download_w = drive.CreateFile({'id': file_id_w})

# Download the two files to a local disc
download_n.GetContentFile('file.csv')
nutrition = pd.read_csv("file.csv")

download_w.GetContentFile('file.csv')
weight = pd.read_csv("file.csv")

# Check all went well
nutrition.head()
#weight.head()

,Date,Meal,Calories,Fat (g),Saturated Fat,Polyunsaturated Fat,Monounsaturated Fat,Trans Fat,Cholesterol,Sodium (mg),Potassium,Carbohydrates (g),Fiber,Sugar,Protein (g),Vitamin A,Vitamin C,Calcium,Iron,Note
0,2020-06-24,Breakfast,523.0,11.0,3.1,2.4,3.7,0.0,372.0,912.0,138.0,74.7,0.0,4.4,25.6,160.0,10.0,58.0,31.8,NaN
1,2020-06-24,Lunch,127.0,7.2,2.1,0.1,0.0,0.0,40.0,423.0,271.0,3.2,0.0,3.5,13.9,20.0,13.0,5.0,22.0,NaN
2,2020-06-24,Dinner,321.0,11.6,2.0,0.0,3.4,0.0,127.0,925.0,164.0,41.9,1.4,1.5,11.0,0.0,0.0,0.0,0.0,NaN
3,2020-06-24,Snacks,525.0,11.5,2.5,3.5,5.0,0.0,2.0,449.0,615.0,101.5,5.4,67.4,6.1,30.0,6.0,17.0,26.0,NaN
4,2020-06-25,Breakfast,264.1,4.3,0.0,0.0,0.0,0.0,12.5,423.9,422.8,39.2,8.1,22.6,19.0,137.3,40.4,147.3,14.0,NaN


First things first I want to form a relationship between these tables where weight from the current day is connected to nutrition from the day before. To do this I'll create a new field "Nutri_Date" that saves just the Year-Month-Day and then reduces the Day by 1.


In [17]:
# Define a function to change the date type and then subtract a day
def convert_date_string(input_string):
    dt = datetime.strptime(input_string, "%b %d %Y %H:%M:%S")
    dt = dt - timedelta(days=1)
    return dt.strftime("%Y-%m-%d")

# Apply the function
weight['Nutri_Date'] = weight['Time of Measurement'].apply(convert_date_string)

# Spot check results
print(weight['Nutri_Date'])

0      2021-01-25
1      2021-01-27
2      2021-01-29
3      2021-01-30
4      2021-02-03
          ...    
895    2023-02-11
896    2023-02-12
897    2023-02-12
898    2023-02-13
899    2023-02-13
Name: Nutri_Date, Length: 900, dtype: object


Next let's merge these two tables into one data table, keeping all values to be able to see which days we have full information and which days have missing information.

In [26]:
# Outter merge the two tables, creating some records with null values
health = pd.merge(weight, nutrition, how='outer', left_on='Nutri_Date', right_on='Date')

# Check things went as I planned
print("The health data set has ", len(health), " records.")
print(health.head())

The health data set has  2641  records.
      Time of Measurement   Weight(lb)   BMI Body Fat(%)  \
0  Jan 26  2021  07:01:32        130.4  24.6        27.5   
1  Jan 26  2021  07:01:32        130.4  24.6        27.5   
2  Jan 26  2021  07:01:32        130.4  24.6        27.5   
3  Jan 26  2021  07:01:32        130.4  24.6        27.5   
4  Jan 28  2021  09:14:30        130.6  24.7        27.6   

  Fat-free Body Weight(lb) Subcutaneous Fat(%) Visceral Fat Body Water(%)  \
0                     94.4                24.9            7          49.7   
1                     94.4                24.9            7          49.7   
2                     94.4                24.9            7          49.7   
3                     94.4                24.9            7          49.7   
4                     94.6                25.0            7          49.7   

  Skeletal Muscle(%) Muscle Mass(lb)  ... Potassium Carbohydrates (g) Fiber  \
0               42.2            88.8  ...     410.2      

Now that we have a master data set named 'health' let's explore our missing values.

In [29]:
# Create data frames that have missing values
missing_nutrition = health[health['Date'].isnull()]
missing_weight = health[health['BMI'].isnull()]

# Explore what percent of our data has missing values
prop_missing_nutri = round(len(missing_nutrition)/len(health)*100,2)
prop_missing_weight = round(len(missing_weight)/len(health)*100,2)

# Print summary of results
print("Of the ", len(health), "records in our health data set ")
print( prop_missing_nutri, "% is missing nutrition data and")
print( prop_missing_weight, "% is missing weight data.")

Of the  2641 records in our health data set 
14.46 % is missing nutrition data and
22.23 % is missing weight data.


I see that about 37% of our data is missing either nutrition or weight data, but that still leaves well over 1000 records that are complete. I'll do my initial analysis on records with complete data, and if time I'll explore imputing missing values to see if this gains any insights.

In [32]:
# Outter merge the two tables, creating some records with null values
health_com = pd.merge(weight, nutrition, how='inner', left_on='Nutri_Date', right_on='Date')

# Look at my columns to decide what I want to explore next
print(health_com.columns)

Index(['Time of Measurement', ' Weight(lb)', 'BMI', 'Body Fat(%)',
       'Fat-free Body Weight(lb)', 'Subcutaneous Fat(%)', 'Visceral Fat',
       'Body Water(%)', 'Skeletal Muscle(%)', 'Muscle Mass(lb)',
       'Bone Mass(lb)', 'Protein (%)', 'BMR(kcal)', 'Metabolic Age', 'Remarks',
       'Nutri_Date', 'Date', 'Meal', 'Calories', 'Fat (g)', 'Saturated Fat',
       'Polyunsaturated Fat', 'Monounsaturated Fat', 'Trans Fat',
       'Cholesterol', 'Sodium (mg)', 'Potassium', 'Carbohydrates (g)', 'Fiber',
       'Sugar', 'Protein (g)', 'Vitamin A', 'Vitamin C', 'Calcium', 'Iron',
       'Note'],
      dtype='object')
